In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName('myApp').setMaster('local[*]'))

In [2]:
import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

In [3]:
def get_bigrams(words, start):
    
    bigrams = []
    
    for i in range(len(words)-1):
        if words[i].lower() == start:
            bigram = '{}_{}'.format(words[i].lower(), words[i+1].lower())
            cnt = 1
            bigrams.append((bigram, cnt))
    
    return bigrams

In [4]:
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
narodnaya_bigrams = wiki.flatMap(lambda item: get_bigrams(item, 'narodnaya'))
bigrams_count = narodnaya_bigrams.reduceByKey(lambda a, b: a + b).sortByKey()
result = bigrams_count.collect()

In [5]:
for bigram, cnt in result:
    print bigram, cnt

narodnaya_gazeta 1
narodnaya_volya 9
